In [1]:
!pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.3 MB/s eta 0:00:00


In [2]:
!pip install cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [3]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [5]:
!pip install --upgrade pinecone-client


In [7]:
# Install Pinecone
!pip install pinecone-client

# Install Cohere (if using it)
!pip install cohere

# Install OpenAI (alternative to Cohere, you can pick one)
!pip install openai

# Install Transformers (for working with language models)
!pip install transformers

# Install other helper libraries like datasets
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00


In [9]:
import os
from pinecone import Pinecone, ServerlessSpec

# Set up your API key and environment
API_KEY = "df8490a2-ccb3-47f4-8ded-c658d28092df"  # Your Pinecone API key
CLOUD_PROVIDER = "aws"
REGION = "us-east-1"
INDEX_NAME = "multilingual-e5-large"

# Initialize Pinecone using the updated method
pc = Pinecone(
    api_key=API_KEY
)

# Check if the index exists, create it if not
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=1024,  # Example dimension size (adjust if needed)
        metric='euclidean',
        spec=ServerlessSpec(
            cloud=CLOUD_PROVIDER,
            region=REGION
        )
    )

# Connect to the index
index = pc.Index(INDEX_NAME)

print(f"Connected to Pinecone index: {INDEX_NAME}")


Connected to Pinecone index: multilingual-e5-large


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load a pre-trained embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to convert text to embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Sample data to add to Pinecone
data = [
    {"id": "1", "text": "Pinecone is a vector database used for fast similarity searches."},
    {"id": "2", "text": "Transformers are deep learning models used for NLP tasks."},
    {"id": "3", "text": "Cohere provides natural language processing models as a service."},
]

# Generate embeddings for each document in your data
for item in data:
    item['embedding'] = get_embedding(item['text'])

# Show the data with embeddings
data


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

[{'id': '1',
  'text': 'Pinecone is a vector database used for fast similarity searches.',
  'embedding': array([-4.90412265e-02, -7.71585345e-01, -4.30068702e-01,  8.63965228e-02,
          1.01588972e-01,  4.06462774e-02, -4.07085493e-02, -4.81403731e-02,
          1.51497573e-01, -9.68304947e-02,  1.60790384e-01,  2.50122458e-01,
         -1.57427788e-01,  6.92545891e-01, -6.66284621e-01,  1.03914820e-01,
         -1.32643953e-01,  4.74163920e-01,  2.96799213e-01,  4.91789766e-02,
         -3.50359738e-01,  2.75917143e-01, -1.58898413e-01,  2.45519310e-01,
          1.82811007e-01, -1.58124059e-01, -5.75847812e-02, -1.75755501e-01,
         -4.86689992e-03, -9.19747204e-02,  6.36634588e-01,  5.57876825e-01,
          4.02934283e-01,  7.91609824e-01, -8.22988749e-01,  2.54310757e-01,
         -1.11080907e-01, -4.87885833e-01, -1.42009571e-01,  2.21137941e-01,
         -3.98826718e-01,  3.92467409e-01,  9.89435893e-03,  2.92077452e-01,
          1.84930354e-01,  4.53054875e-01, -5.495

In [12]:
# Delete the existing index if it exists
if INDEX_NAME in pc.list_indexes().names():
    pc.delete_index(INDEX_NAME)
    print(f"Deleted existing index: {INDEX_NAME}")

# Create a new index with the correct dimension
pc.create_index(
    name=INDEX_NAME,
    dimension=384,  # Set dimension to match the embedding size
    metric='euclidean',
    spec=ServerlessSpec(
        cloud=CLOUD_PROVIDER,
        region=REGION
    )
)
print(f"Created new index: {INDEX_NAME} with dimension 384")

# Reconnect to the new index
index = pc.Index(INDEX_NAME)
print(f"Connected to Pinecone index: {INDEX_NAME}")


Deleted existing index: multilingual-e5-large
Created new index: multilingual-e5-large with dimension 384
Connected to Pinecone index: multilingual-e5-large


In [13]:
# Sample data to add to Pinecone (same data as before)
data = [
    {"id": "1", "text": "Pinecone is a vector database used for fast similarity searches."},
    {"id": "2", "text": "Transformers are deep learning models used for NLP tasks."},
    {"id": "3", "text": "Cohere provides natural language processing models as a service."},
]

# Generate embeddings for each document in your data
for item in data:
    item['embedding'] = get_embedding(item['text'])

# Insert embeddings into Pinecone
vectors = [(item['id'], item['embedding'].tolist()) for item in data]
index.upsert(vectors)

print("Data has been inserted into Pinecone!")


Data has been inserted into Pinecone!


In [18]:
import numpy as np

# Function to query Pinecone with refined format handling
def query_pinecone(query_text):
    # Convert query text to embedding
    query_embedding = get_embedding(query_text)

    # Ensure the embedding has the correct dimension
    if len(query_embedding) != 384:
        raise ValueError(f"Query embedding has an incorrect dimension: {len(query_embedding)}")

    # Convert embedding to a list of floats
    query_embedding_list = query_embedding.astype(float).tolist()

    # Print the first 10 values of the embedding for verification
    print("Query embedding (first 10 values):", query_embedding_list[:10])

    # Perform the search
    try:
        results = index.query(
            vector=query_embedding_list,
            top_k=3  # Number of top results to retrieve
        )
        return results
    except Exception as e:
        print(f"Error querying Pinecone: {e}")
        return None

# Example query
query_text = "What is Pinecone used for?"
results = query_pinecone(query_text)

# Print out the results if successful
if results:
    print("Query results:")
    for match in results['matches']:
        print(f"ID: {match['id']}, Score: {match['score']}")
else:
    print("No results returned.")


Query embedding (first 10 values): [-0.27784329652786255, -0.5503368377685547, -0.26920145750045776, 0.575202465057373, 0.359493225812912, 0.13438507914543152, 0.6109287142753601, 0.34823480248451233, 0.4706628918647766, 0.16040465235710144]
Query results:
ID: 1, Score: 39.1423836
ID: 3, Score: 73.3802338
ID: 2, Score: 78.0385208


In [19]:
# Example additional queries
queries = [
    "What is a transformer model?",
    "How does Cohere provide NLP services?",
    "Explain the usage of vector databases."
]

# Perform queries and print results
for query_text in queries:
    results = query_pinecone(query_text)
    print(f"Results for query '{query_text}':")
    if results:
        for match in results['matches']:
            print(f"ID: {match['id']}, Score: {match['score']}")
    else:
        print("No results returned.")
    print()


Query embedding (first 10 values): [-0.7771531939506531, -0.02776266261935234, -0.07198414951562881, 0.03030281513929367, -0.1056462749838829, -0.002098997123539448, -0.47047969698905945, 0.5864933729171753, 0.3649824857711792, -0.02775442972779274]
Results for query 'What is a transformer model?':
ID: 2, Score: 27.5080414
ID: 3, Score: 56.6780739
ID: 1, Score: 66.2054825

Query embedding (first 10 values): [0.03443841263651848, -0.028277002274990082, -0.20577149093151093, -0.18343722820281982, -0.060011569410562515, 0.2661300003528595, -0.11360406130552292, -0.09707090258598328, 0.2117118090391159, -0.17560364305973053]
Results for query 'How does Cohere provide NLP services?':
ID: 3, Score: 14.9086
ID: 2, Score: 38.0095863
ID: 1, Score: 52.3379898

Query embedding (first 10 values): [0.1565718948841095, -0.10678175836801529, -0.47320446372032166, 0.06638743728399277, 0.10721603780984879, -0.17319485545158386, 0.09599544107913971, 0.1478099226951599, -0.05433483421802521, 0.1189562901

In [20]:
!pip install cohere


In [24]:
import cohere

# Replace with your actual API key
api_key = 'fLxnpWWyrsYKM1CjH9cFrXRhijqt3CLcn7RF9rog'
cohere_client = cohere.Client(api_key)

try:
    # Test a simple generation request
    response = cohere_client.generate(
        model='command-xlarge-nightly',
        prompt="Test if API key is working.",
        max_tokens=10
    )
    print(response.generations[0].text)
except cohere.error.UnauthorizedError as e:
    print("Authorization error:", e)
except Exception as e:
    print("An error occurred:", e)


Testing if an API key is working involves making a


In [25]:
import cohere

# Initialize Cohere client with your API key
api_key = 'fLxnpWWyrsYKM1CjH9cFrXRhijqt3CLcn7RF9rog'
cohere_client = cohere.Client(api_key)

try:
    # Make a simple request to generate text
    response = cohere_client.generate(
        model='command-xlarge-nightly',  # Or another model suitable for your use case
        prompt="Test if API key is working.",
        max_tokens=10
    )
    print("Response:", response.generations[0].text)
except cohere.error.UnauthorizedError as e:
    print("Authorization error:", e)
except Exception as e:
    print("An error occurred:", e)


Response: Testing if an API key is working typically involves making


In [26]:
import cohere

# Initialize Cohere client with your API key
api_key = 'fLxnpWWyrsYKM1CjH9cFrXRhijqt3CLcn7RF9rog'
cohere_client = cohere.Client(api_key)

def generate_answer(prompt):
    try:
        response = cohere_client.generate(
            model='command-xlarge-nightly',  # or another model if preferred
            prompt=prompt,
            max_tokens=150  # adjust based on your needs
        )
        return response.generations[0].text
    except cohere.error.UnauthorizedError as e:
        print("Authorization error:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

# Example usage
query_text = "Explain the usage of vector databases."
answer = generate_answer(query_text)
print("Answer:", answer)


Answer: Vector databases are specialized databases designed to efficiently store, manage, and query high-dimensional vector data, which is commonly used in machine learning and artificial intelligence applications. Vector data refers to arrays of numbers that represent various features or characteristics of objects, such as images, text, audio, or other types of data. These vectors are typically generated by machine learning models, such as word embeddings, image feature extractors, or recommendation systems.

The main purpose of vector databases is to enable fast and scalable similarity search operations on large collections of vectors. Similarity search involves finding the nearest neighbors or the most similar vectors to a given query vector based on a chosen distance metric, such as Euclidean distance or cosine similarity. Vector databases provide indexing and search algorithms


In [27]:
import cohere

# Initialize Cohere client with your API key
api_key = 'fLxnpWWyrsYKM1CjH9cFrXRhijqt3CLcn7RF9rog'
cohere_client = cohere.Client(api_key)

def generate_answer(prompt):
    try:
        response = cohere_client.generate(
            model='command-xlarge-nightly',  # or another model if preferred
            prompt=prompt,
            max_tokens=150  # adjust based on your needs
        )
        return response.generations[0].text
    except cohere.error.UnauthorizedError as e:
        print("Authorization error:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

def handle_query(query_text):
    # Generate an answer based on the query
    answer = generate_answer(query_text)
    return answer

# Example usage
query_text = "Explain the usage of vector databases."
answer = handle_query(query_text)
print("Answer:", answer)


Answer: Vector databases are a type of database specifically designed to store and manage high-dimensional vector data, which is commonly used in machine learning and artificial intelligence applications. Vector data refers to numerical arrays that represent features or embeddings of various entities, such as text documents, images, audio, or any other type of data that can be transformed into a vector representation.

The usage of vector databases revolves around the efficient storage, retrieval, and querying of these vector embeddings. Here are some key aspects and use cases of vector databases:
1. **Similarity Search**: Vector databases enable similarity search or nearest neighbor search. This allows users to find the closest matching vectors to a given query vector. For example, in a text search application, you can find documents that


In [28]:
# Define a list of test queries
test_queries = [
    "What is the purpose of vector databases?",
    "How do vector embeddings work?",
    "Explain similarity search in vector databases.",
    "What are the applications of vector databases?",
    "How does a vector database differ from a traditional database?"
]

# Test the QA bot with the test queries
for query in test_queries:
    answer = handle_query(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}\n")


Query: What is the purpose of vector databases?
Answer: Vector databases are a type of database specifically designed to store, manage, and retrieve high-dimensional vector data, which is commonly used in machine learning and artificial intelligence applications. They are optimized for efficient storage, indexing, and querying of vector embeddings, which are mathematical representations of data points in a continuous vector space.

The main purpose of vector databases is to enable fast and scalable similarity search on large-scale vector datasets. Traditional relational databases are not well-suited for handling high-dimensional vector data, as they are optimized for structured data with well-defined schemas. Vector databases, on the other hand, provide specialized indexing and search algorithms that allow for efficient computation of similarity between vectors.

Here are some key purposes and benefits of vector databases

Query: How do vector embeddings work?
Answer: Vector embeddings

In [29]:
# Define a list of test queries
test_queries = [
    "What is the purpose of vector databases?",
    "How do vector embeddings work?",
    "Explain similarity search in vector databases.",
    "What are the applications of vector databases?",
    "How does a vector database differ from a traditional database?"
]

# Test the QA bot with the test queries
for query in test_queries:
    answer = handle_query(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}\n")


Query: What is the purpose of vector databases?
Answer: Vector databases are specialized database management systems designed to efficiently store, manage, and retrieve high-dimensional vector data, particularly in the context of machine learning and artificial intelligence applications. The primary purpose of vector databases is to enable fast and scalable similarity search on large collections of vectors, which are numerical representations of data points in a high-dimensional space.
**Here are some key purposes and benefits of vector databases:**
1. **Efficient Vector Storage:** Vector databases provide optimized storage formats and indexing techniques tailored for high-dimensional vectors. They can handle sparse and dense vector representations, ensuring efficient use of storage resources.
2. **Similarity Search:** The core functionality of vector databases is to support similarity search operations. Given a query vector, the database can

Query: How do vector embeddings work?
Answ